In [1]:
import os
from diffusers import StableDiffusionPipeline
import torch
from peft import PeftModel
from pathlib import Path

c:\ProgramData\anaconda3\envs\genimg_bovagau\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# 1. Load model gốc và Embedding bạn vừa train xong
base_model_id = "runwayml/stable-diffusion-v1-5"
embedding_path_bo = r"C:\Users\MY LAPTOP\Downloads\bovagau-textual-inversion\learned_embeds-steps-3000-Bo2.safetensors"
# embedding_path_gau = r"C:\Users\MY LAPTOP\Downloads\bovagau-textual-inversion\learned_embeds-steps-3000-Gau.safetensors"

In [46]:

pipeline = StableDiffusionPipeline.from_pretrained(
    base_model_id
)
pipeline.load_textual_inversion(embedding_path_bo)
# pipeline.load_textual_inversion(embedding_path_gau)

Loading weights: 100%|██████████| 196/196 [00:00<00:00, 406.44it/s, Materializing param=text_model.final_layer_norm.weight]
CLIPTextModel LOAD REPORT from: C:\Users\MY LAPTOP\.cache\huggingface\hub\models--runwayml--stable-diffusion-v1-5\snapshots\451f4fe16113bff5a5d2269ed5ad43b0592e9a14\text_encoder
Key                                | Status     |  | 
-----------------------------------+------------+--+-
text_model.embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
Loading weights: 100%|██████████| 396/396 [00:00<00:00, 396.57it/s, Materializing param=visual_projection.weight]
StableDiffusionSafetyChecker LOAD REPORT from: C:\Users\MY LAPTOP\.cache\huggingface\hub\models--runwayml--stable-diffusion-v1-5\snapshots\451f4fe16113bff5a5d2269ed5ad43b0592e9a14\safety_checker
Key                                               | Status     |  | 
-------------------------------------

In [47]:
print("<bo>" in pipeline.tokenizer.get_vocab())
print("<Gau>" in pipeline.tokenizer.get_vocab())


True
False


In [48]:
print("Trump" in pipeline.tokenizer.get_vocab())

False


In [49]:
image = pipeline(
    prompt="Trump wearing a suit",
    negative_prompt="blurry, low quality",
    # guidance_scale=10.0,
    # num_inference_steps=30,
    
    ).images[0]
image.save("output1_0_trump.png")

 22%|██▏       | 11/50 [00:49<02:54,  4.48s/it]


KeyboardInterrupt: 

In [ ]:


image = pipeline(
    prompt="a cartoon drawing of <Gau> wearing a suit",
    negative_prompt="blurry, low quality, realistic",
    # guidance_scale=10.0,
    # num_inference_steps=30,
    
    ).images[0]
image.save("output1_0_normal.png")

100%|██████████| 50/50 [03:13<00:00,  3.87s/it]


## LoRA

In [5]:
output_merged_path = r"C:\Users\MY LAPTOP\OneDrive\Documents\GitHub\Meme-Generator\models\SD-1-5-bo-gau"

In [ ]:


pipe = StableDiffusionPipeline.from_pretrained(base_model_id, torch_dtype=torch.float16)
pipe.load_textual_inversion(embedding_path_bo) # Load <Bo> vào
# pipe.load_textual_inversion(embedding_path_gau) 


# 2. Lưu lại thành một model hoàn toàn mới (đã chứa Bo)
# Model này sẽ hiểu <Bo> là gì mà không cần load file .bin nữa

pipe.save_pretrained(output_merged_path, safe_serialization=True)

print(f"Đã xong! Hãy dùng đường dẫn '{output_merged_path}'.")

Loading weights: 100%|██████████| 196/196 [00:00<00:00, 296.31it/s, Materializing param=text_model.final_layer_norm.weight]
CLIPTextModel LOAD REPORT from: C:\Users\MY LAPTOP\.cache\huggingface\hub\models--runwayml--stable-diffusion-v1-5\snapshots\451f4fe16113bff5a5d2269ed5ad43b0592e9a14\text_encoder
Key                                | Status     |  | 
-----------------------------------+------------+--+-
text_model.embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
Loading weights: 100%|██████████| 396/396 [00:01<00:00, 304.35it/s, Materializing param=visual_projection.weight]
StableDiffusionSafetyChecker LOAD REPORT from: C:\Users\MY LAPTOP\.cache\huggingface\hub\models--runwayml--stable-diffusion-v1-5\snapshots\451f4fe16113bff5a5d2269ed5ad43b0592e9a14\safety_checker
Key                                               | Status     |  | 
-------------------------------------

Đã xong! Hãy dùng đường dẫn 'C:\Users\MY LAPTOP\OneDrive\Documents\GitHub\Meme-Generator\models\SD-1-5-bo-gau' để train LoRA.


In [ ]:
pipeline = StableDiffusionPipeline.from_pretrained(
    output_merged_path,
    torch_dtype=torch.float16
)

# Load LoRA weights into UNet using PEFT
pipeline.unet = PeftModel.from_pretrained(
    pipeline.unet,
    r"C:\Users\MY LAPTOP\Downloads\Bo_final",
    is_trainable=False
)

pipeline = pipeline.to("cuda")

Loading pipeline components...: 100%|██████████| 7/7 [00:02<00:00,  3.02it/s]


In [ ]:
image = pipeline(
    prompt="a cartoon drawing of a <bo> wearing a suit",
    negative_prompt="blurry, low quality, realistic",
    # guidance_scale=10.0,
    # num_inference_steps=30,
    
    ).images[0]
image.save("outputLora.png")

100%|██████████| 50/50 [00:09<00:00,  5.45it/s]


# add Controlnet

In [2]:
from diffusers import (
    StableDiffusionControlNetPipeline, 
    ControlNetModel, 
)
from dwpose import DwposeDetector
from diffusers.utils import load_image

controlnet_model_id = "lllyasviel/control_v11p_sd15_openpose"

c:\ProgramData\anaconda3\envs\genimg_bovagau\Lib\site-packages\dwpose\util2.py:17: UserWarning: USE_SYMLINKS not set successfully. Using default value: False to download models.
  warnings.warn("USE_SYMLINKS not set successfully. Using default value: False to download models.")


In [3]:
print("Loading ControlNet Model...")
controlnet = ControlNetModel.from_pretrained(controlnet_model_id, torch_dtype=torch.float16)

Loading ControlNet Model...


c:\ProgramData\anaconda3\envs\genimg_bovagau\Lib\site-packages\huggingface_hub\utils\_validators.py:202: UserWarning: The `local_dir_use_symlinks` argument is deprecated and ignored in `hf_hub_download`. Downloading to a local directory does not use symlinks anymore.
  warnings.warn(


In [6]:
pipeline = StableDiffusionControlNetPipeline.from_pretrained(
    output_merged_path,
    controlnet=controlnet,
    torch_dtype=torch.float16
)

# Load LoRA weights into UNet using PEFT
pipeline.unet = PeftModel.from_pretrained(
    pipeline.unet,
    r"C:\Users\MY LAPTOP\Downloads\Bo_final",
    is_trainable=False
)

pipeline = pipeline.to("cuda")

Loading pipeline components...: 100%|██████████| 7/7 [00:02<00:00,  3.21it/s]


In [8]:
# --- 5. Extract Pose (ControlNet Input) ---
meme_image  = load_image("Picture1.png").convert("RGB")    # the meme/content reference

print("Loading DWPose estimator...")
pose_estimator = DwposeDetector.from_pretrained_default()
pose_skeleton = pose_estimator(
    meme_image, 
    output_type="pil",  # This is the key setting
    include_body=True,
    include_hands=True, 
    include_face=True
)
print("Pose extracted.")

Loading DWPose estimator...

DWPose: Using yolox_l.onnx for bbox detection and dw-ll_ucoco_384.onnx for pose estimation
DWPose: Caching OpenCV DNN module yolox_l.onnx on cv2.DNN...
DWPose: Caching OpenCV DNN module dw-ll_ucoco_384.onnx on cv2.DNN...
DWPose: Bbox 928.16ms
DWPose: Pose 210.81ms on 1 people

Pose extracted.


In [14]:
# --- 9. Tinh chỉnh ---
prompt = "a cartoon drawing of a <bo> wearing a suit"
negative_prompt = "blurry, low quality, realistic, text, NSFW"


# --- 10. Generate Image ---
# generator = torch.Generator(device="cuda").manual_seed(91)

output_image = pipeline(
    prompt=prompt,
    negative_prompt=negative_prompt,
    image=pose_skeleton,            # ControlNet input
    # num_inference_steps=30,
    # guidance_scale=6.0,
    # generator=generator
).images[0]
output_image.save("outputLoraControlNet.png")


100%|██████████| 50/50 [00:19<00:00,  2.60it/s]
